In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd
import time
import json

options = Options()
options.headless = False

driver = webdriver.Chrome(options=options,service=ChromeService(ChromeDriverManager().install()))

In [107]:
match_id = 'https://www.whoscored.com/Matches/1640761/Live/England-Premier-League-2022-2023-Manchester-City-Manchester-United'
driver.get(match_id)

In [108]:
scriptdiv = driver.find_elements(By.CSS_SELECTOR,'script')

In [111]:
jstring = scriptdiv[-5].get_attribute('innerHTML')
jstring = jstring.replace('\n        require.config.params["args"] = ','')
jstring = jstring.replace('\n','')
jstring = jstring.replace('            ','')
jstring = jstring.replace('{matchId:','{"matchId":')
jstring = jstring.replace('matchCentreData:','"matchCentreData":')
jstring = jstring.replace('formationIdNameMappings','"formationIdNameMappings"')
jstring = jstring.replace('matchCentreEventTypeJson','"matchCentreEventTypeJson"')
jstring = jstring.replace('};','}')
jstring[:1000]

'{"matchId":1640761,"matchCentreData": {"playerIdNameDictionary":{"104749":"Riyad Mahrez","365409":"Julián Álvarez","73084":"Kevin De Bruyne","121774":"Ederson","77464":"Ilkay Gündogan","133569":"Stefan Ortega","122945":"Nathan Aké","315227":"Erling Haaland","313171":"Rúben Dias","128967":"João Cancelo","136741":"Bernardo Silva","355110":"Sergio Gómez","122117":"Aymeric Laporte","297390":"Manuel Akanji","331254":"Phil Foden","113069":"Jack Grealish","8155":"Scott Carson","69778":"Kyle Walker","395692":"Cole Palmer","444791":"Rico Lewis","340105":"Lisandro Martínez","114397":"Fred","88526":"Casemiro","364315":"Antony","69344":"Christian Eriksen","93206":"Raphaël Varane","122366":"Anthony Martial","345303":"Tyrell Malacia","123761":"Bruno Fernandes","5583":"Cristiano Ronaldo","353418":"Diogo Dalot","79554":"David de Gea","100008":"Victor Lindelöf","413211":"Anthony Elanga","404086":"Facundo Pellistri","17708":"Tom Heaton","300299":"Marcus Rashford","346300":"Jadon Sancho","118244":"Luke 

In [168]:
d = json.loads(jstring)
matchData = d['matchCentreData']
matchId = d['matchId']
events = matchData['events']
events[136]

{'id': 2466445047.0,
 'eventId': 95,
 'minute': 7,
 'second': 16,
 'teamId': 167,
 'playerId': 331254,
 'relatedEventId': 94,
 'relatedPlayerId': 136741,
 'x': 91.2,
 'y': 57.5,
 'expandedMinute': 7,
 'period': {'value': 1, 'displayName': 'FirstHalf'},
 'type': {'value': 16, 'displayName': 'Goal'},
 'outcomeType': {'value': 1, 'displayName': 'Successful'},
 'qualifiers': [{'type': {'value': 17, 'displayName': 'BoxCentre'}},
  {'type': {'value': 102, 'displayName': 'GoalMouthY'}, 'value': '52.3'},
  {'type': {'value': 77, 'displayName': 'HighLeft'}},
  {'type': {'value': 72, 'displayName': 'LeftFoot'}},
  {'type': {'value': 103, 'displayName': 'GoalMouthZ'}, 'value': '23.4'},
  {'type': {'value': 178, 'displayName': 'StandingSave'}},
  {'type': {'value': 56, 'displayName': 'Zone'}, 'value': 'Center'},
  {'type': {'value': 55, 'displayName': 'RelatedEventId'}, 'value': '94'},
  {'type': {'value': 154, 'displayName': 'IntentionalAssist'}},
  {'type': {'value': 22, 'displayName': 'RegularP

In [121]:
list(d['matchCentreData'].keys())

['playerIdNameDictionary',
 'periodMinuteLimits',
 'timeStamp',
 'attendance',
 'venueName',
 'referee',
 'weatherCode',
 'elapsed',
 'startTime',
 'startDate',
 'score',
 'htScore',
 'ftScore',
 'etScore',
 'pkScore',
 'statusCode',
 'periodCode',
 'home',
 'away',
 'maxMinute',
 'minuteExpanded',
 'maxPeriod',
 'expandedMinutes',
 'expandedMaxMinute',
 'periodEndMinutes',
 'commonEvents',
 'events',
 'timeoutInSeconds']

In [156]:
player_dict = d['matchCentreData']['playerIdNameDictionary']
players = {
    'id':player_dict.keys(),
    'name':player_dict.values()
}
players_df = pd.DataFrame(players).reset_index()
players_df['id'] = players_df['id'].astype(int)

In [166]:
playerId = 123761
players_df[players_df['id']==playerId]['name'].iloc[0]

'Bruno Fernandes'

In [167]:
event_list = []
for e in enumerate(events):
    ev = e[1]    
    try:
        player_id = ev['playerId']        
    except:
        player_id = None        
    try:
        player_name = players_df[players_df['id']==player_id]['name'].iloc[0]
        print(f'{player_id} : {player_name}')
    except:
        player_name = None
    try:
        endX = ev['endX']
    except:
        endX = None
    try:
        endY = ev['endY']
    except:
        endY = None    
        
    event_list.append({
        'id': ev['id'],
        'eventId': ev['eventId'],
        'minute': ev['minute'],
        'teamId': ev['teamId'],
        'playerId': player_id,
        'playerName':player_name,
        'matchId':matchId,
        'x': ev['x'],
        'y': ev['y'],
        'endX': endX,
        'endY': endY,
        'expandedMinute': ev['expandedMinute'],
        'period': ev['period']['displayName'],
        'type': ev['type']['displayName'],
        'outcomeType': ev['outcomeType']['displayName'],
        'isTouch': ev['isTouch']
    })
event_df2 = pd.DataFrame(event_list)


123761 : Bruno Fernandes
340105 : Lisandro Martínez
345303 : Tyrell Malacia
340105 : Lisandro Martínez
93206 : Raphaël Varane
340105 : Lisandro Martínez
69778 : Kyle Walker
121774 : Ederson
122945 : Nathan Aké
128967 : João Cancelo
121774 : Ederson
297390 : Manuel Akanji
315227 : Erling Haaland
93206 : Raphaël Varane
73084 : Kevin De Bruyne
297390 : Manuel Akanji
122945 : Nathan Aké
121774 : Ederson
122945 : Nathan Aké
128967 : João Cancelo
113069 : Jack Grealish
113069 : Jack Grealish
353418 : Diogo Dalot
353418 : Diogo Dalot
73084 : Kevin De Bruyne
122945 : Nathan Aké
79554 : David de Gea
79554 : David de Gea
345303 : Tyrell Malacia
300299 : Marcus Rashford
364315 : Antony
353418 : Diogo Dalot
122945 : Nathan Aké
122945 : Nathan Aké
77464 : Ilkay Gündogan
122945 : Nathan Aké
315227 : Erling Haaland
113069 : Jack Grealish
136741 : Bernardo Silva
315227 : Erling Haaland
336915 : Scott McTominay
73084 : Kevin De Bruyne
79554 : David de Gea
136741 : Bernardo Silva
336915 : Scott McTomina